# Get Salesforce Data to Join with Marketing Data

## Set up and load data

In [97]:
from toolbox.datalake import DataLake
from dotenv import load_dotenv
import pandas as pd
import datetime
import json
import os

In [6]:
load_dotenv()
CONNECT_STR = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [7]:
dl = DataLake(CONNECT_STR)

In [8]:
BASE_SF = 'Unprocessed/Salesforce/2022/04/14'

In [9]:
leads = dl.get_json_lines_as_df(f'{BASE_SF}/Lead.json')
contacts = dl.get_json_lines_as_df(f'{BASE_SF}/Contact.json')
oppconroles = dl.get_json_lines_as_df(f'{BASE_SF}/OpportunityContactRole.json')
opportunities = dl.get_json_lines_as_df(f'{BASE_SF}/Opportunity.json')
tasks = dl.get_json_lines_as_df(f'{BASE_SF}/Task.json')
events = dl.get_json_lines_as_df(f'{BASE_SF}/Event.json')

## Leads and Contacts

1. Remove unneeded columns.
2. Merge Leads and Contacts to "prospects" table.
3. Coalesce duplicate columns.

In [14]:
req_lead_fields = ['Id', 'Status', 'OwnerId', 'IsConverted', 'ConvertedDate', 'ConvertedAccountId', 'ConvertedContactId', 'ConvertedOpportunityId', 'CreatedDate', 'Chargebee_Plan_Id__c', 'Chargebee_Subscription_Status__c', 'Became_a_marketing_qualified_lead_date__c', 'Account_Name__c', 'HubSpot_Lead_Score__c', 'Account_Number_of_Lost_Opportunities__c', 'Account_Number_of_Open_Opportunities__c', 'Account_Number_of_Opportunities__c', 'Account_Number_of_Won_Opportunities__c', 'Account_has_CS_TS__c', 'Customer_ID__c', 'Subscription_ID__c', 'Subscription_Status__c', 'Subscription_activated_at__c', 'Subscription_cancelled_at__c', 'Subscription_cf_signed_users_count__c', 'Subscription_cf_users_count__c', 'Subscription_current_term_end__c', 'Subscription_trial_start__c', 'Total_Customer_MRR__c']
req_contact_fields = ['Id', 'AccountId', 'OwnerId', 'CreatedDate', 'Chargebee_Plan_Id__c', 'Chargebee_Subscription_Status__c', 'Gtmhub_Edition_ID__c', 'HubSpot_Lead_Score__c', 'Account_Number_of_Lost_Opportunities__c', 'Account_Number_of_Open_Opportunities__c', 'Account_Number_of_Opportunities__c', 'Account_Number_of_Won_Opportunities__c', 'Account_has_CS_TS__c', 'Customer_ID__c', 'Subscription_ID__c', 'Subscription_Status__c', 'Subscription_activated_at__c', 'Subscription_cancelled_at__c', 'Subscription_cf_signed_users_count__c', 'Subscription_cf_users_count__c', 'Subscription_current_term_end__c', 'Subscription_trial_start__c', 'Total_Customer_MRR__c']

In [15]:
leads = leads.loc[:, req_lead_fields]
contacts = contacts.loc[:, req_contact_fields]

In [17]:
leads = leads.rename({'Account_Name__c': 'AccountId'}, axis=1)

In [18]:
prospects = contacts.merge(leads, how='outer', left_on='Id', right_on='ConvertedContactId')

In [22]:
# Get matching columns
x_cols = [x for x in prospects.columns if x.endswith('_x')]
y_cols = [x for x in prospects.columns if x.endswith('_y')]
matching = {}
for k in x_cols:
    for v in y_cols:
        if k.replace('_x', '') == v.replace('_y', ''):
            matching[k] = v
            break
        else:
            continue

# Combine matching columns into one column
for i in matching.keys():
    col = i.replace('_x', '')
    prospects.loc[:, col] = prospects[i].combine_first(prospects[matching[i]])
    if i.endswith('_x'):
        prospects = prospects.drop([i, matching[i]], axis=1)
    else:
        prospects = prospects.drop([matching[i]], axis=1)

In [24]:
prospects.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288893 entries, 0 to 288892
Data columns (total 30 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Gtmhub_Edition_ID__c                       594 non-null     object 
 1   Status                                     249422 non-null  object 
 2   IsConverted                                249422 non-null  object 
 3   ConvertedDate                              726 non-null     object 
 4   ConvertedAccountId                         724 non-null     object 
 5   ConvertedContactId                         724 non-null     object 
 6   ConvertedOpportunityId                     602 non-null     object 
 7   Became_a_marketing_qualified_lead_date__c  65696 non-null   object 
 8   Id                                         288893 non-null  object 
 9   AccountId                                  165118 non-null  object 
 10  OwnerId 

## Opportunities & OpportunityContactRoles

1. Remove unneeded columns.
2. Merge opportunity data on contact relationship data.

In [26]:
opportunities = opportunities.loc[:, ['Id', 'AccountId', 'RecordTypeId', 'IsClosed', 'IsWon', 'Amount', 'CloseDate', 'Type', 'OwnerId']]

In [28]:
oppconroles = oppconroles.loc[:, ['Id', 'OpportunityId', 'ContactId']]

In [29]:
opps = oppconroles.merge(opportunities, how='left', left_on='OpportunityId', right_on='Id')

In [31]:
opps = opps.rename({
    'Id_x': 'OppConRoleId'
}, axis=1).drop(['Id_y'], axis=1)

In [32]:
opps

,OppConRoleId,OpportunityId,ContactId,AccountId,RecordTypeId,IsClosed,IsWon,Amount,CloseDate,Type,OwnerId
0,00K8a00000XJm4bEAD,0068a00001GdcT8AAJ,0038a00002rQon6AAC,0018a00001nrAeLAAU,0124W0000007bPpQAI,False,False,100000.0,2022-12-29T00:00:00,New Business,0054W00000Ce45HQAR
1,00K8a00000XJniXEAT,0068a00001GdeqOAAR,0038a00002rQxZzAAK,0018a00001nrB2gAAE,0124W0000007bPpQAI,False,False,50000.0,2022-06-30T00:00:00,None,0054W00000E5aXmQAJ
2,00K8a00000XJpSvEAL,0068a00001Gdi6IAAR,0038a00002rR8K4AAK,0018a00001nrBWdAAM,0124W0000007bPpQAI,False,False,50000.0,2022-06-30T00:00:00,New Business,0058a00000INI4UAAX
3,00K8a00000XJjFgEAL,0068a00001GdX03AAF,0038a00002qcCjNAAU,0018a00001nr6rJAAQ,0124W0000007bPpQAI,False,False,15000.0,2022-08-31T00:00:00,New Business,0058a00000INI4ZAAX
4,00K8a00000XJjX1EAL,0068a00001GdXOyAAN,0038a00002qcD6RAAU,0018a00001nr6teAAA,0124W0000007bPpQAI,False,False,50000.0,2022-08-26T00:00:00,New Business,0054W00000D8NSsQAN
...,...,...,...,...,...,...,...,...,...,...,...
1583,00K8a00000Vtg2pEAB,0068a00001GjkgKAAR,0038a00002qLgMSAA0,0014W00002Fh9PwQAJ,0124W0000007bPuQAI,False,False,642600.0,2025-03-31T00:00:00,Existing Business,0058a00000INIFDAA5
1584,00K8a00000Vtg2oEAB,0068a00001GjkgKAAR,0038a00002oIoybAAC,0014W00002Fh9PwQAJ,0124W0000007bPuQAI,False,False,642600.0,2025-03-31T00:00:00,Existing Business,0058a00000INIFDAA5
1585,00K8a00000Vtg2nEAB,0068a00001GjkgKAAR,0034W00002d7kJRQAY,0014W00002Fh9PwQAJ,0124W0000007bPuQAI,False,False,642600.0,2025-03-31T00:00:00,Existing Business,0058a00000INIFDAA5
1586,00K8a00000VtgMjEAJ,0068a00001GjlBhAAJ,0038a00002qLgMSAA0,0014W00002Fh9PwQAJ,0124W0000007bPuQAI,False,False,59200.0,2022-03-31T00:00:00,Existing Business,0058a00000INIFDAA5


## Get supplemental prospect data

1. Remove excess tasks.
2. Update opps with OpportunityType.
3. Create summary dataframes.
4. Merge sumary dataframes with prospects.
5. Clean up fields.
6. Output to CSV.

In [34]:
tasks = tasks[
    (tasks.RecordTypeId == '0124W000001lKRjQAM') & # Only sales tasks record type
    (tasks.Status == 'Completed') &
    (tasks.Type.isin(['Call', 'Email', 'Other']))
] 

In [91]:
def opp_record_type(row):
    if row['RecordTypeId'] == '0124W0000007bPpQAI':
        return 'New Business'
    elif row['RecordTypeId'] == '0124W0000007bPuQAI':
        return 'Renewal'
    else:
        return 'Partner'

def opp_type(row):
    if row['RecordType'] == 'Renewal' or row['RecordType'] == 'Partner':
        return row['RecordType']
    if row['Type'] == 'Existing Business':
        return 'Upsell'
    else:
        return 'New Business'

In [46]:
opps.loc[:, 'RecordType'] = opps.apply(opp_record_type, axis=1)
opps.loc[:, 'OpportunityType'] = opps.apply(opp_type, axis=1)

In [79]:
opps_to_merge = opps.groupby(['ContactId', 'OpportunityType']).size().unstack(fill_value=0).reset_index().rename({'New Business': 'new_business_count', 'Upsell': 'upsell_count', 'Renewal': 'renewal_count', 'Partner': 'partner_count'}, axis=1)

In [80]:
tasks_to_merge = tasks.groupby(['WhoId', 'Type']).size().unstack(fill_value=0).reset_index().rename({'Call': 'call_count', 'Email': 'email_count', 'Other': 'other_count'}, axis=1)

In [81]:
events_to_merge = events.groupby(['WhoId']).size().reset_index(name='event_count')

In [83]:
prospects2 = prospects.merge(opps_to_merge, how='left', left_on='Id', right_on='ContactId').drop(['ContactId'], axis=1)

In [87]:
prospects3 = prospects2.merge(tasks_to_merge, how='left', left_on='Id', right_on='WhoId').drop(['WhoId'], axis=1)

In [112]:
prospects4 = prospects3.merge(events_to_merge, how='left', left_on='Id', right_on='WhoId').drop(['WhoId'], axis=1)

In [113]:
# Clean up
prospects4.HubSpot_Lead_Score__c = prospects4.HubSpot_Lead_Score__c.fillna(0)
prospects4.Account_Number_of_Lost_Opportunities__c = prospects4.Account_Number_of_Lost_Opportunities__c.fillna(0)
prospects4.Account_Number_of_Open_Opportunities__c = prospects4.Account_Number_of_Open_Opportunities__c.fillna(0)
prospects4.Account_Number_of_Opportunities__c = prospects4.Account_Number_of_Opportunities__c.fillna(0)
prospects4.Account_Number_of_Won_Opportunities__c = prospects4.Account_Number_of_Won_Opportunities__c.fillna(0)
prospects4.Total_Customer_MRR__c = prospects4.Total_Customer_MRR__c.fillna(0)
prospects4.new_business_count = prospects4.new_business_count.fillna(0)
prospects4.renewal_count = prospects4.renewal_count.fillna(0)
prospects4.upsell_count = prospects4.upsell_count.fillna(0)
prospects4.partner_count = prospects4.partner_count.fillna(0)
prospects4.call_count = prospects4.call_count.fillna(0)
prospects4.email_count = prospects4.email_count.fillna(0)
prospects4.other_count = prospects4.other_count.fillna(0)
prospects4.event_count = prospects4.event_count.fillna(0)
prospects4.Subscription_cf_signed_users_count__c = prospects4.Subscription_cf_signed_users_count__c.fillna(0)
prospects4.Subscription_cf_users_count__c = prospects4.Subscription_cf_users_count__c.fillna(0)

prospects4.ConvertedDate = prospects4.ConvertedDate.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S') if pd.notnull(x) else None)
prospects4.Became_a_marketing_qualified_lead_date__c = prospects4.Became_a_marketing_qualified_lead_date__c.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S') if pd.notnull(x) else None)
prospects4.CreatedDate = prospects4.CreatedDate.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S') if pd.notnull(x) else None)
prospects4.Subscription_activated_at__c = prospects4.Subscription_activated_at__c.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S') if pd.notnull(x) else None)
prospects4.Subscription_current_term_end__c = prospects4.Subscription_current_term_end__c.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S') if pd.notnull(x) else None)
prospects4.Subscription_trial_start__c = prospects4.Subscription_trial_start__c.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S') if pd.notnull(x) else None)

In [117]:
prospects4.to_csv('salesforce_prospects_data.csv', index=False)